# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering
algorithm to group pixels by similar spectral signatures. **k-means** is
an **exploratory** method for finding patterns in data. Because it is
unsupervised, you don’t need any training data for the model. You also
can’t measure how well it “performs” because the clusters will not
correspond to any particular land cover class. However, we expect at
least some of the clusters to be identifiable as different types of land
cover.

You will use the [harmonized Sentinel/Landsat multispectral
dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf).
You can access the data with an [Earthdata
account](https://www.earthdata.nasa.gov/learn/get-started) and the
[`earthaccess` library from
NSIDC](https://github.com/nsidc/earthaccess):

## STEP 1: Set up

### Step 1a: Load libraries and set GDAL parameters

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Import all libraries you will need for this analysis</li>
<li>Configure GDAL parameters to help avoid connection errors:
<code>python      os.environ["GDAL_HTTP_MAX_RETRY"] = "5"      os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"</code></li>
</ol></div></div>

In [1]:
### Import libraries

# Create reproducible file paths
import os

# Serializing and unserializing objects (save objects to disk and load later)
import pickle

# Regular expressions
import re

# Give warnings
import warnings

# Project coordinate systems for spatial data and mapping
import cartopy.crs as ccrs

# Access satellite imagery through the NASA API
import earthaccess

# Spatial data analysis
import earthpy as et

# Work with vector/shapefiles
import geopandas as gpd

# Vizualization tools
import geoviews as gv
import hvplot.pandas
import hvplot.xarray

# Work with arrays
import numpy as np

# Work with tabular data
import pandas as pd

# Work with raster data
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr

# Use progress bar
from tqdm.notebook import tqdm
from ipywidgets import IntProgress
from IPython.display import display

# Polygons
from shapely.geometry import Polygon

# Use kmeans clustering
from sklearn.cluster import KMeans

### set GDAL (Geospatial Data Abstraction Library) parameters
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

### don't show non-critical warnings
warnings.simplefilter('ignore')

c:\Users\nymve\miniconda3\envs\earth-analytics-python\Lib\site-packages\earthpy\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


### Step 1b: Run the caching decorator

Below you can find code for a caching **decorator** which you can use in
your code. To use the decorator:

``` python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the
`do_something()` function, and only run the code if the results don’t
already exist. To override the caching, for example temporarily after
making changes to your code, set `override=True`. Note that to use the
caching decorator, you must write your own function to perform each
task!

You might notice that typically in these assignments, we start by creating a data_dir to store our data files. Here, our caching decorator is making the data directory for us.

In [2]:
### make the caching decorator
def cached(func_key, override=False): # Add =True if wanting to rerun
    """
    A decorator to cache function results
    
    Parameters
    ==========
    key: str
      File basename used to save pickled results
    override: bool
      When True, re-compute even if the results are already stored
    """
    def compute_and_cache_decorator(compute_function):
        """
        Wrap the caching function
        
        Parameters
        ==========
        compute_function: function
          The function to run and cache results
        """
        def compute_and_cache(*args, **kwargs):
            """
            Perform a computation and cache, or load cached result.
            
            Parameters
            ==========
            args
              Positional arguments for the compute function
            kwargs
              Keyword arguments for the compute function
            """
            ### Add an identifier from the particular function call
            if 'cache_key' in kwargs:
                key = '_'.join((func_key, kwargs['cache_key']))
            else:
                key = func_key

            ### define a file path based on the directory structure in earthpy
            path = os.path.join(
                
                ### earthpy directory
                et.io.HOME, 
                
                ### earthpy dataset
                et.io.DATA_NAME, 
                
                ### make a subdirectory called "jars"
                'jars', 
                
                ### use f-string (formatted string) to create a string by embedding the value
                ### of the variable "key" into the string 
                ### use .pickle file extension (a pickle file is a serialized python objecT)
                f'{key}.pickle')
            
            ### Check if the cache exists already or if we should override caching
            if not os.path.exists(path) or override:
                
                ### Make jars directory if needed
                os.makedirs(os.path.dirname(path), exist_ok=True)
                
                ### Run the compute function as the user did
                result = compute_function(*args, **kwargs)
                
                ### Pickle the object (save to file)
                ### open the file at filename (wb - write binary)
                with open(path, 'wb') as file:
                    
                    ### save the result without needing to recompute when loading
                    ### it back into Python
                    pickle.dump(result, file)
            
            ### if the file already exists/we are not overriding the cache
            else:
               
                ### Unpickle the object (load the cached result) (rb - read binary)
                with open(path, 'rb') as file:
                    
                    ### use pickle.load to unserialize the file back into a python object
                    result = pickle.load(file)
                    
            return result
        
        return compute_and_cache
    
    return compute_and_cache_decorator

## STEP 2: Study site

For this analysis, you will use a watershed from the [Water Boundary
Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products),
HU12 watersheds (WBDHU12.shp).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Download the Water Boundary Dataset for region 8 (Mississippi)</li>
<li>Select watershed 080902030506</li>
<li>Generate a site map of the watershed</li>
</ol>
<p>Try to use the <strong>caching decorator</strong></p></div></div>

We chose this watershed because it covers parts of New Orleans an is
near the Mississippi Delta. Deltas are boundary areas between the land
and the ocean, and as a result tend to contain a rich variety of
different land cover and land use types.

In [3]:
# Assign the hydrologic unit code
HUC_LEVEL = 12

# Download, unzip, and read shapefile, using cache decorator
@cached(f'wbd_08_hu{HUC_LEVEL}_gdf')

# Make a function to read in the file
def read_wbd_file(wbd_filename, cache_key):

    # Define the URL we're pulling data from
    wbd_url = (

        ### add base URL
        "https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/"

        # Insert the name of the specific file we want
        f"{wbd_filename}.zip"
    )

    # Download data and unzip it into the directory
    wbd_dir = et.data.get_data(url = wbd_url)

    # Path to shapefile in the fir
    wbd_path = os.path.join(wbd_dir,
                            'Shape',                            
                            f'WBDHU{HUC_LEVEL}.shp')
    
    # Read the shp as gdf
    wbd_gdf = gpd.read_file(wbd_path,
                            
                            # Use pyogrio library
                            engine = 'pyogrio')
    
    # Give us the gdf for the watershed boundary
    return wbd_gdf

In [4]:
# open the shapefile using the read_wbd_file function that we created
wbd_gdf = read_wbd_file("WBD_08_HU2_Shape",
                        f'hu{HUC_LEVEL}')

In [5]:
# Look at data
wbd_gdf

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID,geometry
0,{8AFB1AF9-7296-4303-89DE-14CD073B859A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"535297,540579",29441.81,119.15,LA,...,Gourd Bayou-Youngs Bayou,S,"LE,ID,DD",080500011308,0.0,0.0,NaN,NaN,1,"POLYGON ((-92.00021 32.53586, -91.99994 32.535..."
1,{916A17A6-B4A0-4FD7-9BB8-FFD1936B15B2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,535512,11406.67,46.16,LA,...,Hams Creek,S,ID,080802050104,0.0,0.0,NaN,NaN,2,"POLYGON ((-93.37574 30.58982, -93.3747 30.5891..."
2,{493C7EC1-2F1C-4B84-AFFB-6F6868A9868E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547190,559640",29138.21,117.92,LA,...,Caney Creek-Bayou D'Arbonne,S,NM,080402060503,0.0,0.0,NaN,NaN,3,"POLYGON ((-93.07761 32.88752, -93.07784 32.887..."
3,{49A3C087-B460-4F97-9D99-78CBB675248B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77417,78285",17759.39,71.87,AR,...,L'Aigle Creek-Saline River,S,NM,080402020206,0.0,0.0,NaN,NaN,4,"POLYGON ((-92.08947 33.29383, -92.0897 33.2938..."
4,{0FB41498-11EA-4AB1-AF05-E2A8E5E2E274},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,1628466,98564.62,398.88,LA,...,West Cote Blanche Bay,W,NM,080801030800,0.0,0.0,NaN,NaN,5,"POLYGON ((-91.62408 29.73947, -91.62195 29.737..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,{9E524E78-2605-48CB-A41F-618AFCDF513D},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"695171,692611",9132.46,36.96,MS,...,Widow Bayou-Big Sunflower River,S,"AW,LE,TF",080302071707,0.0,0.0,NaN,NaN,2637,"POLYGON ((-90.76273 32.97428, -90.76209 32.973..."
2637,{C11913D9-C534-4755-884C-4CAD470ED143},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77280,81842",20587.50,83.31,AR,...,Lindermans Lake-Bayou Des Arc,S,LE,080203010306,0.0,0.0,NaN,NaN,2638,"POLYGON ((-91.73427 34.99197, -91.7342 34.9923..."
2638,{3EEBF422-01AC-4322-A63C-24C0A34E1E4F},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,714675,21405.46,86.62,MO,...,Brewer Lake,S,"LE,DD,IT,TF",080103000102,0.0,0.0,NaN,NaN,2639,"POLYGON ((-89.13715 36.97285, -89.13387 36.970..."
2639,{4734715C-0F4A-4211-BBAE-86605B20B79A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547104,553857",38848.09,157.21,LA,...,Blounts Creek-Calcasieu River,S,ID,080802030302,0.0,0.0,NaN,NaN,2640,"POLYGON ((-92.75965 31.12593, -92.75881 31.125..."


In [6]:
### Filter the shapefile to the specific watershed we're using

# Define the gdf for the watershed by subsetting the gdf of the whole watershed dataset
delta_gdf = wbd_gdf[wbd_gdf[
                
    # Filter the gdf to the row(s) with the watershe we want 
    # Use "dissolve" to merge the geometries of all the rows matching the target watershed            
    f'huc{HUC_LEVEL}'].isin(['080902030506'])].dissolve() 



### check it out
delta_gdf

,geometry,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,...,huc12,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID
0,"POLYGON ((-89.97047 29.74687, -89.96593 29.750...",{E942B72E-599E-48F5-908A-EA5265701C14},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"536881,539539",37355.86,151.17,...,080902030506,Manuel Canal-Spanish Lake,D,GC,080902030508,0.0,0.0,NaN,NaN,2560


In [7]:
### Make a site map with satellite imagery in the background
(
    # Project the delta_gdf to Mercator
    delta_gdf.to_crs(ccrs.Mercator())

    # Use hvplot
    .hvplot(

        # Make the watershed transparent
        alpha = 0.40, fill_color = "white",

        # Add satellite basemap
        tiles = 'EsriImagery',

        crs = ccrs.Mercator())

        # Set plot size
        .opts(width = 600, height = 300)
)


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

The watershed site for this analysis is in the Mississippi Delta to the southeast outskirts of New Orleans, Louisiana. The watershed boundary was determined from the USGS National Watershed Boundary dataset at the Hydrologic Unit Code (HUC) 12 - 080902030506. This area is experiencing rapid land loss, salt water intrusion, and wetland degradation (USGS, 2011). New Orleans was originally built on a wetland habitat and is a below sea level community on a delta. It experiences unique challenges that come from coastal and deltaic hydrologic processes including flooding and sediment deposits (Day et al., 2007). The city and area is experiencing significant water-related threats due to climate change, and currently depends on levees and pumps for its survival (Day et al., 2021).

<u>Data citation:</u>
U.S. Geological Survey, 2024, National watershed boundary (HUC12) dataset for the conterminous United States, retrieved [02/06/2026], from https://data.usgs.gov/datacatalog/data/USGS:63d31e73d34e06fef1501265. 

## STEP 3: Multispectral data

### Step 3a: Search for data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Log in to the <code>earthaccess</code> service using your Earthdata
credentials:
<code>python      earthaccess.login(persist=True)</code></li>
<li>Modify the following sample code to search for granules of the
HLSL30 product overlapping the watershed boundary from May to October
2023 (there should be 76 granules):
<code>python      results = earthaccess.search_data(          short_name="...",          cloud_hosted=True,          bounding_box=tuple(gdf.total_bounds),          temporal=("...", "..."),      )</code></li>
</ol></div></div>

In [8]:
### Log in to earthaccess
earthaccess.login(persist = True)

In [9]:
### Search for HLS granules we want
results = earthaccess.search_data(

    ### Specify which dataset and spatial resolution we want 
    short_name = "HLSL30",

    ### Specify that we're using cloud data
    cloud_hosted = True,

    ### Use the bounding box from our watershed boundary
    bounding_box = tuple(delta_gdf.total_bounds),

    ### set the temporal range of the data
    temporal = ("2024-06", "2024-08")
)

In [10]:
results

[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -89.79864173, 'Latitude': 29.70347853}, {'Longitude': -89.76643746, 'Latitude': 30.69278312}, {'Longitude': -90.91181412, 'Latitude': 30.71627038}, {'Longitude': -90.93262544, 'Latitude': 29.72659663}, {'Longitude': -89.79864173, 'Latitude': 29.70347853}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z', 'EndingDateTime': '2024-06-07T16:31:11.509Z'}}
 Size(MB): 169.50417041778564
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SAA.tif', 'https://data.l

### Step 3b: Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the
granules you are downloading and make sure they line up with your
shapefile. You could also use a DataFrame, dictionary, or a custom
object to store this information.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each search result:
<ol type="1">
<li>Get the following information (HINT: look at the [‘umm’] values for
each search result):
<ul>
<li>granule id (UR)</li>
<li>datetime</li>
<li>geometry (HINT: check out the shapely.geometry.Polygon class to
convert points to a Polygon)</li>
</ul></li>
<li>Open the granule files. I recommend opening one granule at a time,
e.g. with (<code>earthaccess.open([result]</code>).</li>
<li>For each file (band), get the following information:
<ul>
<li>file handler returned from <code>earthaccess.open()</code></li>
<li>tile id</li>
<li>band number</li>
</ul></li>
</ol></li>
<li>Compile all the information you collected into a GeoDataFrame</li>
</ol></div></div>

In [11]:
### Make a function to process all the granules from the earthaccess search
### And extract information for each granule

# Define the function
def get_earthaccess_links(results):

    # Make and display a progress bar
    f = IntProgress(min = 0, max = len(results), description = 'Open granules')
    display(f)

    # Use a regular expression to extract tile_id and bank from .tif files
    url_re = re.compile(
        r'\.(?P<tile_id>\w+)\.\d+T\d+\.v\d\.\d\.(?P<band>[A-Za-z0-9]+)\.tif')

    # Accumulate gdf rows from each granule
    link_rows = []

    # Loop over granules to extract info
    for granule in results:

        # Locate metadata (UMM = universal metadata model)
        info_dict = granule['umm']

        # Pull out unique identifier for the granule
        granule_id = info_dict['GranuleUR']

        # Extract date/time 
        datetime = pd.to_datetime(
            info_dict['TemporalExtent']['RangeDateTime']['BeginningDateTime']
        )

        # Extact boundary coordinates for granule
        points = (
            info_dict
            ['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]
            ['Boundary']['Points']
        )

        # Make polygon using coordinate points for granule
        geometry = Polygon(
            [(point['Longitude'],
              point['Latitude']) for point in points]
        )

        # Get url and open granule
        files = earthaccess.open([granule])

        # Loop through each file in the granule
        for file in files:

            # Use url regular expression to get url
            match = url_re.search(file.full_name)

            # If match is found, append data to link_rows gdf we initialized
            if match is not None: 
                link_rows.append(

                    # Makes a gdf with the granule's data and geometry
                    gpd.GeoDataFrame(
                        dict(

                        # Timestamp
                        datetime = [datetime],

                        # Unique ID
                        tile_id = [match.group('tile_id')],

                        # Spectral band name
                        band = [match.group('band')],

                        # url
                        url = [file],

                        geometry = [geometry]
                    ),

                    # Set crs
                    crs ="EPSG:4326"
                )
            )

        # Update progress bar after each granule is done
        f.value += 1

    # Combine into a single gdf   
    file_df = pd.concat(link_rows).reset_index(drop = True)

    # Return the final gdf file
    return file_df

In [12]:
granule = results[0]

granule

Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -89.79864173, 'Latitude': 29.70347853}, {'Longitude': -89.76643746, 'Latitude': 30.69278312}, {'Longitude': -90.91181412, 'Latitude': 30.71627038}, {'Longitude': -90.93262544, 'Latitude': 29.72659663}, {'Longitude': -89.79864173, 'Latitude': 29.70347853}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z', 'EndingDateTime': '2024-06-07T16:31:11.509Z'}}
Size(MB): 169.50417041778564
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.VZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B06.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B09.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B04.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B03.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B07.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.VAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.Fmask.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B01.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B11.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B05.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B02.tif']

In [13]:
info_dict = granule['umm']
info_dict

{'TemporalExtent': {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z',
   'EndingDateTime': '2024-06-07T16:31:11.509Z'}},
 'GranuleUR': 'HLS.L30.T15RYP.2024159T163111.v2.0',
 'AdditionalAttributes': [{'Name': 'LANDSAT_PRODUCT_ID',
   'Values': ['LC08_L1TP_022039_20240607_20240607_02_RT']},
  {'Name': 'CLOUD_COVERAGE', 'Values': ['10']},
  {'Name': 'MGRS_TILE_ID', 'Values': ['15RYP']},
  {'Name': 'SPATIAL_COVERAGE', 'Values': ['100']},
  {'Name': 'SPATIAL_RESOLUTION', 'Values': ['30.0']},
  {'Name': 'SPATIAL_RESAMPLING_ALG', 'Values': ['Cubic Convolution']},
  {'Name': 'HLS_PROCESSING_TIME', 'Values': ['2024-06-21T18:26:46Z']},
  {'Name': 'SENSING_TIME', 'Values': ['2024-06-07T16:31:11.5093050Z']},
  {'Name': 'HORIZONTAL_CS_NAME', 'Values': ['UTM, WGS84, UTM ZONE 15']},
  {'Name': 'ULX', 'Values': ['699960.0']},
  {'Name': 'ULY', 'Values': ['3400020.0']},
  {'Name': 'ADD_OFFSET', 'Values': ['0']},
  {'Name': 'REF_SCALE_FACTOR', 'Values': ['0.0001']},
  {'Name': 'THERM_SC

In [14]:
### Run the function to get granule search results
file_df = get_earthaccess_links(results)

IntProgress(value=0, description='Open granules', max=44)

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

...

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
# Check to make sure its a geodataframe
type(file_df)

geopandas.geodataframe.GeoDataFrame

### Step 3c: Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your
results using the `cached` decorator or by writing your own caching
code. I also recommend testing this step with one or two dates before
running the full computation.

This code should include at least one **function** including a
numpy-style docstring. A good place to start would be a function for
opening a single masked raster, applying the appropriate scale
parameter, and cropping.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each granule:
<ol type="1">
<li><p>Open the Fmask band, crop, and compute a quality mask for the
granule. You can use the following code as a starting point, making sure
that <code>mask_bits</code> contains the quality bits you want to
consider: ```python # Expand into a new dimension of binary bits bits =
( np.unpackbits(da.astype(np.uint8), bitorder=‘little’)
.reshape(da.shape + (-1,)) )</p>
<p># Select the required bits and check if any are flagged mask =
np.prod(bits[…, mask_bits]==0, axis=-1) ```</p></li>
<li><p>For each band that starts with ‘B’:</p>
<ol type="1">
<li>Open the band, crop, and apply the scale factor</li>
<li>Name the DataArray after the band using the <code>.name</code>
attribute</li>
<li>Apply the cloud mask using the <code>.where()</code> method</li>
<li>Store the DataArray in your data structure (e.g. adding a
GeoDataFrame column with the DataArray in it. Note that you will need to
remove the rows for unused bands)</li>
</ol></li>
</ol></li>
</ol></div></div>

## ADD docstrings

In [16]:
### Apply cached decorator to function
@cached('delta_reflectance_da_df')

# Write function that computes reflectance data using 
# Search results (df of urls) and watershed boundary
def compute_reflectance_da(search_results, boundary_gdf):

    """Connect to files using VSI, crop them, apply a cloud mask, and wrangle

    Return a single reflectance Dataframe with bands as columns
    and centroid coordinates and dattime as the index
    
    Parameters
    ==========
    search_results:list
        Search result links to the files (urls)
    boundary_gdf: gpd.GeoDataFrame
        Boundary used to crop the data
    """

    # Write a function to open raster from url, apply scale factor, and crop and mask data
    def open_dataarray(url, boundary_proj_gdf, scale = 1, masked = True):

        # Open raster data
        da = rxr.open_rasterio(url, masked = masked).squeeze() * scale  

        # Reproject the boundary needed to match the raster crs
        if boundary_proj_gdf is None:
            boundary_proj_gdf = boundary_gdf.to_crs(da.rio.crs)

        # Crop the raster to the bounding box
        cropped = da.rio.clip_box(*boundary_proj_gdf.total_bounds)

        # Return the cropped image of that tile
        return cropped

    # Write function to apply a cloud mask
    def compute_quality_mask(da, mask_bits = [1, 2, 3]):
        
        """Mask out low quality data by bit"""
        
        # Unpack the bits to a new axis  
        bits = (

            # Unpack each number into individual bits
            np.unpackbits(

                # Convert to 8-bit unsigned integrer format
                da.astype(np.uint8),

                # Set the order of the bits
                bitorder = "little"
            # Reshampe to match original data with an extra dimension for the bits
            ).reshape(da.shape +(-1, ))
        )
        
        # Grab bits we want and check if their flagged
        mask = np.prod(

            # Open bits
            bits[
                ...,
                            mask_bits] == 0,
                            axis = -1)


        # Return the mask
        return mask
    
    # Grab metadata
    file_df = get_earthaccess_links(search_results)

    # Store results for each granule
    granule_da_rows = []

    # Store projected boundary
    boundary_proj_gdf = None

    # Group the data by each granule
    group_iter = file_df.groupby(

        # Datetime and tile_id
        ['datetime', 'tile_id'])
    
    # loop through each image and its metadata
    for (datetime, tile_id), granule_df in tqdm(group_iter):

        # Print status bar
        print(f'Processing granule {tile_id} {datetime}')

        # Find each granule's cloud mask file (fmask) url
        cloud_mask_url = (
            granule_df.loc[granule_df.band == 'Fmask', 'url']
            .values[0])
        
        # Open granule cloud cover
        cloud_masked_cropped_da = open_dataarray(cloud_mask_url, boundary_proj_gdf, scale = 1, masked = False)

        # Compute the cloud mask
        cloud_mask = compute_quality_mask(cloud_masked_cropped_da)
        
        # Loop through each spectral band to open, crop, and mask the band
        da_list = []
        df_list = []
        
        # Lop through each band in the granule
        for i, row in granule_df.iterrows():

            # Only loop through the spectral bands
            if row.band.startswith('B'):

                # Open band's raster and scale to reflectance
                band_cropped = open_dataarray(
                    row.url, boundary_proj_gdf, scale = 0.0001)
                
                # Name the raster by the band
                band_cropped.name = row.band

                # Apply the cloud mask to the raster
                row['da'] = band_cropped.where(cloud_mask)

                # Append the row to the granule_da_rows
                granule_da_rows.append(row.to_frame().T)


    # Reassemble the metadata df
    return pd.concat(granule_da_rows)

In [17]:
### apply the function
reflectance_da_df = compute_reflectance_da(results, delta_gdf)

In [18]:
### check out the dataframe
reflectance_da_df

,datetime,tile_id,band,url,geometry,da
45,2024-06-07 16:31:11.509000+00:00,T15RYN,B04,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-89.82661214 28.80213717, -89.795837...",[[<xarray.DataArray 'B04' ()> Size: 4B\narray(...
48,2024-06-07 16:31:11.509000+00:00,T15RYN,B06,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-89.82661214 28.80213717, -89.795837...",[[<xarray.DataArray 'B06' ()> Size: 4B\narray(...
49,2024-06-07 16:31:11.509000+00:00,T15RYN,B03,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-89.82661214 28.80213717, -89.795837...",[[<xarray.DataArray 'B03' ()> Size: 4B\narray(...
50,2024-06-07 16:31:11.509000+00:00,T15RYN,B07,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-89.82661214 28.80213717, -89.795837...",[[<xarray.DataArray 'B07' ()> Size: 4B\narray(...
51,2024-06-07 16:31:11.509000+00:00,T15RYN,B02,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-89.82661214 28.80213717, -89.795837...",[[<xarray.DataArray 'B02' ()> Size: 4B\narray(...
...,...,...,...,...,...,...
640,2024-08-26 16:31:51.172000+00:00,T16RBU,B10,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-90.10082051 29.70587294, -89.295300...",[[<xarray.DataArray 'B10' ()> Size: 4B\narray(...
641,2024-08-26 16:31:51.172000+00:00,T16RBU,B07,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-90.10082051 29.70587294, -89.295300...",[[<xarray.DataArray 'B07' ()> Size: 4B\narray(...
642,2024-08-26 16:31:51.172000+00:00,T16RBU,B04,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-90.10082051 29.70587294, -89.295300...",[[<xarray.DataArray 'B04' ()> Size: 4B\narray(...
643,2024-08-26 16:31:51.172000+00:00,T16RBU,B01,"<File-like object HTTPFileSystem, https://data...","POLYGON ((-90.10082051 29.70587294, -89.295300...",[[<xarray.DataArray 'B01' ()> Size: 4B\narray(...


### Step 3d: Merge and Composite Data

You will notice for this watershed that:   
1. The raster data for each date are spread across 4 granules  
2. Any given image is incomplete because of clouds

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">

*   1. For each band:  
    *   a. For each date:  
        *   i. Merge all 4 granules  
        *   ii. Mask any negative values created by interpolating from the nodata value of -9999 (`rioxarray`) should account for this, but doesn't appear to when merging. If you leave these values in, they will create problems later on
    *   b. Concatenate the merged DataArrays along a new date dimension  
    *   c. Take the mean in the date dimension to create a composite image that fills cloud gaps  
    *   d. Add the band as a dimensions, and give the DataArray a name  
*   2. Concatenate along the band dimension


In [19]:
### Apply cache decorator
@cached('delta_reflectance_da')

# Create a function to merge and composite reflectance data from multiple granules
# End result: single, composite reflectance image for each spectral band
def merge_and_composite_arrays(granule_da_df):
    
    # Initialize a list to store composites after procesing
    da_list = []

    ### loop over each spectral band
    for band, band_df in granule_da_df.groupby('band'):

        # List for storing merged data arrays (one per date)
        merged_das = []

        # Loop over date/time of image acquisition and merge granules for each date
        for dataetime, date_df in band_df.groupby('datetime'):

            # Merge granules for each date
            merged_da = rxrmerge.merge_arrays(list(date_df.da))
           
            # Mask negative values (could be no data or invalid data)
            merged_da = merged_da.where(merged_da > 0)
            
            # Append to merged_das list we initialized
            merged_das.append(merged_da)
            
        # Composite images across dates
        composite_da = xr.concat(merged_das,
                                 
                                 # Make a datetime dimension
                                 # Calculate median value across the dateimtes for the pixel
                                dim = 'datetime').median('datetime')
        
        # Assign band number to attribute of composite data array
        composite_da['band'] = int(band[1:])

        # Name the composite data array
        composite_da.name = 'reflectance'

        # Add processed and composite data array to lsit
        da_list.append(composite_da)

    # Concatenates composite data arrays for each band along band dimension
    return xr.concat(da_list, dim = 'band')

In [20]:
# Call function to get final composite reflectance data 
reflectance_da = merge_and_composite_arrays(reflectance_da_df)

In [21]:
# Check out the data array
reflectance_da

<xarray.DataArray 'reflectance' (band: 10, y: 556, x: 624)> Size: 14MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]],
      shape=(10, 556, 624), dtype=float32)
Coordinates:
  * band         (band) int64 80B 1 2 3 4 5 6 7 9 10 11
  * y            (y) float64 4kB 3.304e+06 3.304e+06 ... 3.287e+06 3.287e+06
  * x            (x) float64 5kB 7.926e+05 7.926e+05 ... 8.112e+05 8.113e+05
    spatial_ref  int64 8B 0

## STEP 4: K-means clustering

Cluster your data by spectral signature using the k-means algorithm.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Convert your DataArray into a <strong>tidy</strong> DataFrame of
reflectance values (hint: check out the <code>.to_dataframe()</code> and
<code>.unstack()</code> methods)</li>
<li>Filter out all rows with no data (all 0s or any N/A values)</li>
<li>Fit a k-means model. You can experiment with the number of groups to
find what works best.</li>
</ol></div></div>

In [22]:
### Convert spectral DataArray to a tidy DataFrame
model_df = (reflectance_da
            
            # Flatten the array into a long dataframe
            .to_dataframe()
            
            # Select the reflectance column
            .reflectance
            
            # Make the table wide: each row will be pixel location
            # And each column is a spectral band with the reflectance value
            .unstack('band')
)

# Check dataframe
model_df

band                        1   2   3   4   5   6   7   9   10  11
y            x                                                    
3.303783e+06 792568.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             792598.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             792628.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             792658.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             792688.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
...                         ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
3.287133e+06 811138.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             811168.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             811198.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             811228.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
             811258.062907 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN

[346944 rows x 10 columns]

In [23]:
# Filter out rows with no data
model_df = model_df.drop(columns = [10,11]).dropna()

# Check the dataframe without NaN
model_df

band                              1       2       3       4       5       6  \
y            x                                                                
3.303783e+06 810148.062907  0.01560  0.0225  0.0409  0.0366  0.0478  0.0281   
             810178.062907  0.01895  0.0256  0.0396  0.0413  0.0426  0.0284   
             810208.062907  0.01915  0.0246  0.0387  0.0377  0.0384  0.0273   
             810238.062907  0.02040  0.0247  0.0440  0.0445  0.0629  0.0418   
             810268.062907  0.01815  0.0245  0.0437  0.0444  0.0618  0.0397   
...                             ...     ...     ...     ...     ...     ...   
3.287163e+06 793768.062907  0.02650  0.0345  0.0548  0.0427  0.0218  0.0098   
             793798.062907  0.02790  0.0351  0.0549  0.0439  0.0221  0.0104   
             793828.062907  0.02580  0.0331  0.0534  0.0419  0.0194  0.0080   
             793858.062907  0.02570  0.0326  0.0521  0.0402  0.0182  0.0064   
             793888.062907  0.02550  0.0340  0.0541  0.0423  0.0199  0.0083   

band                             7       9  
y            x                              
3.303783e+06 810148.062907  0.0181  0.0006  
             810178.062907  0.0220  0.0006  
             810208.062907  0.0236  0.0007  
             810238.062907  0.0266  0.0007  
             810268.062907  0.0259  0.0008  
...                            ...     ...  
3.287163e+06 793768.062907  0.0074  0.0007  
             793798.062907  0.0076  0.0008  
             793828.062907  0.0059  0.0009  
             793858.062907  0.0046  0.0007  
             793888.062907  0.0060  0.0007  

[317917 rows x 8 columns]

In [24]:
# Check the data mins and max
min_values = model_df.min()
max_values = model_df.max()

# Print min/max values
print(min_values)
print(max_values)

band
1    0.0003
2    0.0021
3    0.0100
4    0.0075
5    0.0005
6    0.0006
7    0.0010
9    0.0002
dtype: float32
band
1    0.31425
2    0.35160
3    0.42865
4    0.44420
5    0.59500
6    0.49645
7    0.34510
9    0.00125
dtype: float32


Now we're reading to fit the k-means clustering model. We can run the fit and prediction functions at the same time because we don't have target data.

In [37]:
# Initialize k-means model 
k_means = KMeans(n_clusters = 5)

# Fit model and predict
prediction = k_means.fit_predict(model_df.values)

# Add the predicted values back to the model dataframe
model_df['clusters'] = prediction
model_df

band                              1       2       3       4       5       6  \
y            x                                                                
3.303783e+06 810148.062907  0.01560  0.0225  0.0409  0.0366  0.0478  0.0281   
             810178.062907  0.01895  0.0256  0.0396  0.0413  0.0426  0.0284   
             810208.062907  0.01915  0.0246  0.0387  0.0377  0.0384  0.0273   
             810238.062907  0.02040  0.0247  0.0440  0.0445  0.0629  0.0418   
             810268.062907  0.01815  0.0245  0.0437  0.0444  0.0618  0.0397   
...                             ...     ...     ...     ...     ...     ...   
3.287163e+06 793768.062907  0.02650  0.0345  0.0548  0.0427  0.0218  0.0098   
             793798.062907  0.02790  0.0351  0.0549  0.0439  0.0221  0.0104   
             793828.062907  0.02580  0.0331  0.0534  0.0419  0.0194  0.0080   
             793858.062907  0.02570  0.0326  0.0521  0.0402  0.0182  0.0064   
             793888.062907  0.02550  0.0340  0.0541  0.0423  0.0199  0.0083   

band                             7       9  clusters  
y            x                                        
3.303783e+06 810148.062907  0.0181  0.0006         0  
             810178.062907  0.0220  0.0006         0  
             810208.062907  0.0236  0.0007         0  
             810238.062907  0.0266  0.0007         1  
             810268.062907  0.0259  0.0008         1  
...                            ...     ...       ...  
3.287163e+06 793768.062907  0.0074  0.0007         0  
             793798.062907  0.0076  0.0008         0  
             793828.062907  0.0059  0.0009         0  
             793858.062907  0.0046  0.0007         0  
             793888.062907  0.0060  0.0007         0  

[317917 rows x 9 columns]

## STEP 5: Plot

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Create a plot that shows the k-means clusters next to an RGB image of
the area. You may need to brighten your RGB image by multiplying it by
10. The code for reshaping and plotting the clusters is provided for you
below, but you will have to create the RGB plot yourself!</p>
<p>So, what is <code>.sortby(['x', 'y'])</code> doing for us? Try the
code without it and find out.</p></div></div>

In [38]:
# Make data array with bands to use for rgb: red, green, and blue
rgb = reflectance_da.sel(band = [4, 3, 2])

In [39]:
# Plot rbg
(
    rgb.hvplot.rgb(y = 'y',
                   x = 'x',
                   bands = 'band',
                   data_aspect = 1,
                   xaxis = None,
                   yaxis = None)
)

:RGB   [x,y]   (R,G,B)

In [40]:
# Stretch values
rgb_uint8 = (rgb * 255).astype(np.uint8).where(rgb != np.nan)

# Look at values
rgb_uint8

<xarray.DataArray 'reflectance' (band: 3, y: 556, x: 624)> Size: 4MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]],
      shape=(3, 556, 624), dtype=float32)
Coordinates:
  * band         (band) int64 24B 4 3 2
  * y            (y) float64 4kB 3.304e+06 3.304e+06 ... 3.287e+06 3.287e+06
  * x            (x) float64 5kB 7.926e+05 7.926e+05 ... 8.112e+05 8.113e+05
    spatial_ref  int64 8B 0

In [41]:
# Plot rbg from 0-255
(
    rgb_uint8.hvplot.rgb(y = 'y',
                   x = 'x',
                   bands = 'band',
                   data_aspect = 1,
                   xaxis = None,
                   yaxis = None)
)

:RGB   [x,y]   (R,G,B)

In [42]:
# Enhance brightness
rgb_uint8_bright = rgb_uint8 * 10

# Look at values
rgb_uint8_bright

<xarray.DataArray 'reflectance' (band: 3, y: 556, x: 624)> Size: 4MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]],
      shape=(3, 556, 624), dtype=float32)
Coordinates:
  * band         (band) int64 24B 4 3 2
  * y            (y) float64 4kB 3.304e+06 3.304e+06 ... 3.287e+06 3.287e+06
  * x            (x) float64 5kB 7.926e+05 7.926e+05 ... 8.112e+05 8.113e+05
    spatial_ref  int64 8B 0

In [43]:
# Plot brighter rgb
(
    rgb_uint8_bright.hvplot.rgb(y = 'y',
                   x = 'x',
                   bands = 'band',
                   data_aspect = 1,
                   xaxis = None,
                   yaxis = None)
)

:RGB   [x,y]   (R,G,B)

In [44]:
# Cap saturation
rgb_sat = rgb_uint8_bright.where(rgb_uint8_bright < 255, 255)

In [45]:
# Replot saturated aspect
(
    rgb_sat.hvplot.rgb(y = 'y',
                   x = 'x',
                   bands = 'band',
                   data_aspect = 1,
                   xaxis = None,
                   yaxis = None)
)

:RGB   [x,y]   (R,G,B)

In [46]:
# Plot the clusters from kmeans
(
    model_df.clusters.to_xarray().hvplot(
        x = 'x',
        y = 'y',
        data_aspect = 1,
        xaxis = None,
        yaxis = None)
)

:Image   [x,y]   (clusters)

In [47]:
# Plot the clusters from kmeans, deal with sorting
(
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        x = 'x',
        y = 'y',
        data_aspect = 1,
        xaxis = None,
        yaxis = None)
)

:Image   [x,y]   (clusters)

In [50]:
### plot the k-means clusters
(
     rgb_sat.hvplot.rgb(y = 'y',
                   x = 'x',
                   bands = 'band',
                   data_aspect = 1,
                   xaxis = None,
                   yaxis = None,
                   title = 'True Color Reflectance Composite (RGB)')
    + 
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        cmap="Colorblind", aspect='equal', title = 'Spectral Reflectance Clusters (K-Means)', data_aspect = 1) 
)

:Layout
   .RGB.I   :RGB   [x,y]   (R,G,B)
   .Image.I :Image   [x,y]   (clusters)

## ADD labels titles and plot and interpretation

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Don’t forget to interpret your plot!</p></div></div>

### **K-Means Land Cover Classification in Missippi Delta Watershed in Summer 2024**

Currently, New Orleans is one of the cities in the U.S. most vulnerable to impacts from climate change. This is in part due to the normal deltaic processes, such as subsidence, associated with being built on a river delta. New Orleans relies on engineering, through pumps and levees, to keep the city above water. Deltas rely on natural deposits of sediment to maintain their elevation. Due to these flooding interventions, the city is sinking. New Orleans experiences rapid land loss at an incredible rate of wetland loss the size of a football field every 100 minutes (Restore the Mississippi Delta, n.d.). This process is accelerated by climate change impacts such as increasing storm intensity (including hurricanes), faster sea-level rise, and other factors. Coastal and deltaic areas are important economic hubs as ports support the region and the nation. Ports have historically been important for economic growth and commerce; New Orleans is no exception (Day et al., 2021). The area was historically a significant wetland habitat, which has also been lost and degraded significantly due to development, land use, and the interruption of natural river hydrologic processes.

​This analysis used unsupervised machine learning, K-means, to look at land cover classifications in the watershed on the Mississippi Delta (HUC12 - 080902030506). Five K-Means clusters were used for this analysis. This area has significant wetland and saltwater intrusions, so we would expect those to be clustered. We can see some clustering that may be showing differences as the reflectance data for salt and fresh water. The salt water intrusions might be visible here, , with perhaps yellow being fresh water and dark blue being seawater. We also see clustering of what maybe land, wetlands, agriculture, development, and perhaps active erosion areas. The pink might be wetlands scattered throughout. In the RGB imagery, the water might appear brown due to high levels of suspended sediments in the water. The watershed is on the Mississippi River Delta, so we would expect a high sediment load. These wetlands and the deltaic watershed are very important for ecological health, biodiversity, and supporting fishing economies. In future analysis, it would be interesting to compare a time series of k-means to see any land cover classification differences. 


<u>Data Citation</u>
Masek, J., Ju, J., Roger, J.-C., Skakun, S., Vermote, E., Claverie, M., Dungan, J., Yin, Z., Freitag, B., &amp; Justice, C. (2021). <i>HLS Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0</i> [Data set]. NASA Land Processes Distributed Active Archive Center. https://doi.org/10.5067/HLS/HLSL30.002 Date Accessed: 2026-02-07

<u>Citations</u>

- Day, J. W., Jr., Boesch, D. F., Clairain, E. J., Kemp, G. P., Laska, S. B., Mitsch, W. J., Mashriqui, H., Reed, D. J., Shabman, L., Simenstad, C. A., Streever, B. J., Twilley, R. R., Watson, C. C., Wells, J. T., & Whigham, D. F. (2007). Restoration of the Mississippi Delta: Lessons from Hurricanes Katrina and Rita. Science, 315(5819), 1679–1684. https://doi.org/10.1126/science.1137030

- Day, J. W., Hunter, R., Kemp, G. P., Moerschbaecher, M., & Brantley, C. G. (2021). The “problem” of New Orleans and diminishing sustainability of Mississippi River management — future options. Water, 13(6), 813. https://doi.org/10.3390/w13060813

- Restore the Mississippi River Delta Coalition. (n.d). Land loss in the Mississippi River Delta. Retrieved February 6, 2026, from https://mississippiriverdelta.org/our-coastal-crisis/land-loss/

- U.S. Geological Survey. (2011, November 11). The Mississippi River Delta Basin. U.S. Department of the Interior. Retrieved February 11, 2026, from https://eros.usgs.gov/media-gallery/image-of-the-week/the-mississippi-river-delta-basin